# Cleaning up raw data downloaded as csv from Kaggle

### To do list
- look into scraping up data for missing/nan values for this dataset
- clean data
- make decisions on database tables and what they will contain
- create individual pd dataframes for each table
- use sqlalchemy engine to send dfs to_sql
- figure out how to do the same with mongodb for shits and gigs

In [24]:
import pandas as pd
from sqlalchemy import create_engine

#### Import CSV and convert to pandas dataframe

In [25]:
raw_df = pd.read_csv('winemag-data-130k-v2.csv')
raw_df.head(3)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


In [26]:
raw_df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [27]:
wine_df = raw_df.drop(columns=['Unnamed: 0', 'taster_twitter_handle', 'region_1','region_2', 'designation', ])
wine_df.head(3)

,country,description,points,price,province,taster_name,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


In [28]:
wine_df.isna().sum()

country           63
description        0
points             0
price           8996
province          63
taster_name    26244
title              0
variety            1
winery             0
dtype: int64

In [29]:
wine_df.head(20)

,country,description,points,price,province,taster_name,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,Blackberry and raspberry aromas show a typical...,87,15.0,Northern Spain,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,Italy,"Here's a bright, informal red that opens with ...",87,16.0,Sicily & Sardinia,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
7,France,This dry and restrained wine offers spice in p...,87,24.0,Alsace,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,Germany,Savory dried thyme notes accent sunnier flavor...,87,12.0,Rheinhessen,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel
9,France,This has great depth of flavor with its fresh ...,87,27.0,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam


In [30]:
wine_df['index_col'] = wine_df.index

In [31]:
wine_df['index_col'] = wine_df['index_col'] + 1 

In [32]:
wine_df.head()

,country,description,points,price,province,taster_name,title,variety,winery,index_col
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,1
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,3
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,4
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,5


In [33]:
wine_df.columns

Index(['country', 'description', 'points', 'price', 'province', 'taster_name',
       'title', 'variety', 'winery', 'index_col'],
      dtype='object')

In [34]:
location = wine_df[['index_col', 'country', 'province']]
review = wine_df[['index_col', 'description', 'points', 'taster_name']]
id = wine_df[['index_col', 'title', 'variety', 'winery', 'price']]

In [37]:
# id.head()

,index_col,title,variety,winery,price
0,1,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,NaN
1,2,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,15.0
2,3,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,14.0
3,4,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,13.0
4,5,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,65.0


In [38]:
# review.head()

,index_col,description,points,taster_name
0,1,"Aromas include tropical fruit, broom, brimston...",87,Kerin O’Keefe
1,2,"This is ripe and fruity, a wine that is smooth...",87,Roger Voss
2,3,"Tart and snappy, the flavors of lime flesh and...",87,Paul Gregutt
3,4,"Pineapple rind, lemon pith and orange blossom ...",87,Alexander Peartree
4,5,"Much like the regular bottling from 2012, this...",87,Paul Gregutt


In [39]:
# location.head()

,index_col,country,province
0,1,Italy,Sicily & Sardinia
1,2,Portugal,Douro
2,3,US,Oregon
3,4,US,Michigan
4,5,US,Oregon


In [ ]:
# rds_connection_string = "root:8sc15854@127.0.0.1:3306/<database name>"
# engine = create_engine(f'mysql://{rds_connection_string}')

In [ ]:
# location.to_sql(name='location', con=engine, if_exists='replace', index=False)
# review.to_sql(name='review', con=engine, if_exists='replace', index=False)
# id.to_sql(name='id', con=engine, if_exists='replace', index=False)